In [1]:
#!pip install XlsxWriter


In [2]:
# custom_widgets.py

from PyQt5.QtWidgets import QLabel, QLineEdit, QWidget, QHBoxLayout, QCheckBox, QMenu
from PyQt5.QtCore import Qt

class CustomLabel(QLabel):
    def __init__(self, cell_id, history_callback, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.cell_id = cell_id
        self.history_callback = history_callback
        self.setContextMenuPolicy(Qt.CustomContextMenu)
        self.customContextMenuRequested.connect(self.onCustomContextMenu)
        
    def onCustomContextMenu(self, pos):
        menu = QMenu(self)
        action_history = menu.addAction("Historial de canvis")
        selectedAction = menu.exec_(self.mapToGlobal(pos))
        if selectedAction == action_history:
            self.history_callback(self.cell_id)

class CustomLineEdit(QLineEdit):
    def __init__(self, cell_id, associate_callback, pending_callback, associated_callback, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.cell_id = cell_id
        self.associate_callback = associate_callback
        self.pending_callback = pending_callback
        self.associated_callback = associated_callback
        self.setPlaceholderText("Introdueix nova info")
        self.setStyleSheet("background-color: #FFCCCC; color: #990000;")
        self.setContextMenuPolicy(Qt.CustomContextMenu)
        self.customContextMenuRequested.connect(self.onCustomContextMenu)
        
    def onCustomContextMenu(self, pos):
        menu = QMenu(self)
        action_associate = menu.addAction("Associar documentació al canvi")
        action_pending = menu.addAction("Accedir al gestor de documents pendents")
        action_associated = menu.addAction("Veure documents associats")
        selectedAction = menu.exec_(self.mapToGlobal(pos))
        if selectedAction == action_associate:
            self.associate_callback(self.cell_id)
        elif selectedAction == action_pending:
            self.pending_callback(self.cell_id)
        elif selectedAction == action_associated:
            self.associated_callback(self.cell_id)

class DoubleViewWidget(QWidget):
    def __init__(self, official_text, cell_id, selection_callback, associate_callback,
                 history_callback, pending_callback, associated_callback, parent=None):
        super().__init__(parent)
        self.cell_id = cell_id
        
        self.label = CustomLabel(cell_id, history_callback, official_text)
        self.line_edit = CustomLineEdit(cell_id, associate_callback, pending_callback, associated_callback)
        self.line_edit.textChanged.connect(self.on_text_changed)
        self.checkbox = QCheckBox()
        self.checkbox.toggled.connect(lambda selected: selection_callback(cell_id, selected))
        
        layout = QHBoxLayout()
        layout.setContentsMargins(0, 0, 0, 0)
        layout.addWidget(self.label)
        layout.addWidget(self.line_edit)
        layout.addWidget(self.checkbox)
        self.setLayout(layout)
        
        # Estil per defecte (fons blanc)
        self.default_style = "background-color: white;"
        # Variables d'estat:
        # is_pending = True si hi ha documents pendents (fons taronja)
        # is_associated = True si hi ha modificacions (text o documents associats) i no hi ha pendents
        self.is_pending = False
        self.is_associated = False
        
        self.update_background()
    
    def set_pending(self, pending):
        """Actualitza l'estat pendent i torna a calcular el fons."""
        self.is_pending = pending
        self.update_background()
        
    def set_associated(self, associated):
        """Actualitza l'estat associat (modificacions pendents) i torna a calcular el fons."""
        self.is_associated = associated
        self.update_background()
    
    def on_text_changed(self, text):
        # Cada vegada que canvia el text es torna a calcular el fons
        self.update_background()
    
    def update_background(self):
        """
        Determina el color de fons:
          - Taronja (#FFA500) si hi ha documents pendents.
          - Marró clar (#F5DEB3) si no hi ha pendents però hi ha text (o documents associats).
          - Blanc si no hi ha cap modificació.
        """
        if self.is_pending:
            bg = "#FFA500"
        elif self.line_edit.text().strip() or self.is_associated:
            bg = "#F5DEB3"
        else:
            bg = "white"
        self.label.setStyleSheet(f"background-color: {bg};")
    
    def update_official(self, new_text):
        self.label.setText(new_text)
        self.line_edit.clear()
        # Després d'actualitzar, es consideren eliminades les modificacions pendents
        self.is_associated = False
        self.is_pending = False
        self.update_background()


In [3]:
# excel_handler.py

import pandas as pd
from PyQt5.QtWidgets import QTableWidgetItem, QFileDialog
#from custom_widgets import DoubleViewWidget  # Assegura't que aquest mòdul està disponible

def load_excel_and_populate_table(table, group_info, main_window_instance):
    options = QFileDialog.Options()
    file_name, _ = QFileDialog.getOpenFileName(
        main_window_instance,
        "Selecciona un fitxer Excel",
        "",
        "Excel Files (*.xlsx *.xls)",
        options=options
    )
    if file_name:
        # Carrega el DataFrame original i guarda una còpia en original_excel_df
        df = pd.read_excel(file_name)
        main_window_instance.original_excel_df = df.copy()
        
        df = df.fillna("")
        df_sorted = df.sort_values(list(df.columns)).reset_index(drop=True)
        n_rows, n_cols = df_sorted.shape
        
        table.setRowCount(n_rows)
        table.setColumnCount(n_cols)
        table.setHorizontalHeaderLabels([str(col) for col in df.columns])
        
        for row in range(n_rows):
            for col in range(n_cols):
                value = str(df_sorted.iat[row, col])
                table.setItem(row, col, QTableWidgetItem(value))
        
        for col in range(n_cols):
            group_keys = list(df.columns[:col+1])
            groups = df_sorted.groupby(group_keys, sort=False)
            for key, group in groups:
                indices = group.index.tolist()
                if len(indices) >= 1 and (max(indices) - min(indices) + 1 == len(indices)):
                    start_row = indices[0]
                    span = len(indices)
                    if span > 1:
                        table.setSpan(start_row, col, span, 1)
                    current_text = str(df_sorted.iat[start_row, col])
                    cell_id = f"{start_row}-{col}"
                    dv_widget = DoubleViewWidget(
                        current_text,
                        cell_id=cell_id,
                        selection_callback=main_window_instance.onSelectionChanged,
                        associate_callback=main_window_instance.associateDocumentFromLineEdit,
                        history_callback=main_window_instance.showHistoryFromLabel,
                        pending_callback=main_window_instance.showPendingDocumentsFromLineEdit,
                        associated_callback=main_window_instance.showAssociatedDocumentsFromLineEdit
                    )
                    table.takeItem(start_row, col)
                    table.setCellWidget(start_row, col, dv_widget)
                    group_info.append({
                        "row": start_row,
                        "col": col,
                        "indices": indices,
                        "widget": dv_widget
                    })

def refresh_table_from_df(main_window_instance, df):
    """
    Actualitza el QTableWidget de la MainWindow utilitzant el DataFrame 'df'.
    Aquesta funció es pot cridar per reconstruir la taula quan es carrega
    la sessió desada (per exemple, a partir de les dades originals).
    """
    table = main_window_instance.table
    table.clear()
    if df.empty:
        return
    table.setRowCount(len(df))
    table.setColumnCount(len(df.columns))
    table.setHorizontalHeaderLabels(list(df.columns))
    for row in range(len(df)):
        for col in range(len(df.columns)):
            value = str(df.iat[row, col])
            table.setItem(row, col, QTableWidgetItem(value))


In [4]:
# pending_documents_manager.py

import datetime
import os
from PyQt5.QtWidgets import QDialog, QVBoxLayout, QListWidget, QPushButton, QInputDialog, QFileDialog, QMessageBox, QDialogButtonBox
from PyQt5.QtGui import QBrush, QColor

def update_pending_cell_background(main_window_instance, cell_id):
    """
    Actualitza el fons de la cel·la tenint en compte:
      - Taronja (#FFA500) si hi ha documents pendents.
      - Marró clar (#F5DEB3) si no hi ha pendents però hi ha documents associats o text.
      - Blanc en absència de modificació.
    Si la cel·la conté un widget que tingui els mètodes set_pending i set_associated,
    s'actualitza l'estat del widget.
    
    NOTA: Els valors de cell_id provenen de group_info (files de dades sense la fila de filtres).
    Per tant, la fila real de la taula és row + 1.
    """
    try:
        row, col = map(int, cell_id.split('-'))
    except Exception:
        return

    # La fila real de la taula és la fila del registre + 1 (fila 0 és la fila de filtres)
    data_row = row + 1

    pending_list = main_window_instance.pendingDocuments.get(cell_id, [])
    associated_list = main_window_instance.associatedDocuments.get(cell_id, [])
    
    if pending_list:
        color = QColor("#FFA500")  # Forçar taronja si hi ha documents pendents
    elif associated_list:
        color = QColor("#F5DEB3")  # Marró clar si hi ha documents associats
    else:
        widget = main_window_instance.table.cellWidget(data_row, col)
        if widget and hasattr(widget, "line_edit") and widget.line_edit.text().strip():
            color = QColor("#F5DEB3")
        else:
            color = QColor("white")
    
    # Actualitza el fons del QTableWidgetItem, si existeix
    item = main_window_instance.table.item(data_row, col)
    if item:
        item.setBackground(QBrush(color))
    
    widget = main_window_instance.table.cellWidget(data_row, col)
    if widget:
        if hasattr(widget, "set_pending"):
            widget.set_pending(True if pending_list else False)
        if not pending_list and hasattr(widget, "set_associated"):
            extra = associated_list or (hasattr(widget, "line_edit") and widget.line_edit.text().strip())
            widget.set_associated(True if extra else False)
        # Si hi ha documents pendents, no cridem update_background per evitar sobreescriptura del color taronja
        if not pending_list and hasattr(widget, "update_background"):
            widget.update_background()
        # Força el styleSheet amb el color definit
        widget.setStyleSheet(f"background-color: {color.name()};")


def show_pending_documents(main_window_instance, cell_id):
    """
    Mostra el diàleg de documents pendents per al camp indicat.
    Aquesta funció s'ha de cridar exclusivament des d'una casella de text (QLineEdit).
    """
    dialog = QDialog(main_window_instance)
    dialog.setWindowTitle("Documents pendents per al camp " + cell_id)
    layout = QVBoxLayout(dialog)
    listWidget = QListWidget()
    
    # Mostra els documents pendents associats al cell_id
    pending_list = main_window_instance.pendingDocuments.get(cell_id, [])
    if pending_list:
        for doc in pending_list:
            listWidget.addItem(doc.get("description", "Sense descripció"))
    else:
        listWidget.addItem("No hi ha documents pendents")
    layout.addWidget(listWidget)
    
    btnAdd = QPushButton("Afegir document nou")
    layout.addWidget(btnAdd)
    btnAdd.clicked.connect(lambda: add_pending_document(main_window_instance, cell_id, listWidget))
    
    btnDelete = QPushButton("Eliminar document pendent")
    layout.addWidget(btnDelete)
    def onDelete():
        selectedItems = listWidget.selectedItems()
        if not selectedItems:
            QMessageBox.warning(dialog, "Error", "Selecciona un element per eliminar.")
            return
        doc_desc = selectedItems[0].text()
        if cell_id in main_window_instance.pendingDocuments:
            new_list = [doc for doc in main_window_instance.pendingDocuments[cell_id]
                        if doc.get("description", "Sense descripció") != doc_desc]
            if len(new_list) != len(main_window_instance.pendingDocuments[cell_id]):
                main_window_instance.pendingDocuments[cell_id] = new_list
                QMessageBox.information(dialog, "Èxit", f"Document pendent '{doc_desc}' eliminat.")
            else:
                QMessageBox.warning(dialog, "Error", f"No s'ha trobat l'element '{doc_desc}' per eliminar.")
        listWidget.clear()
        pending_list = main_window_instance.pendingDocuments.get(cell_id, [])
        if pending_list:
            for doc in pending_list:
                listWidget.addItem(doc.get("description", "Sense descripció"))
        else:
            listWidget.addItem("No hi ha documents pendents")
        update_pending_cell_background(main_window_instance, cell_id)
    btnDelete.clicked.connect(onDelete)
    
    listWidget.itemClicked.connect(lambda item: associate_pending_document(main_window_instance, item.text(), cell_id, dialog))
    dialog.setLayout(layout)
    dialog.exec_()


def add_pending_document(main_window_instance, cell_id, listWidget):
    """
    Afegeix un document pendent. Es recull la descripció i s'obté la llista dels camps seleccionats
    en el moment d'invocar-lo (main_window_instance.selectedChanges). El document pendent s'afegeix a totes
    aquestes caselles.
    """
    from PyQt5.QtWidgets import QInputDialog, QMessageBox
    description, ok = QInputDialog.getText(main_window_instance, "Afegir Document Pendent",
                                             "Introdueix la descripció del document pendent:")
    if not ok or not description.strip():
        return
    description = description.strip()
    selected_fields = list(main_window_instance.selectedChanges)
    if cell_id not in selected_fields:
        selected_fields.append(cell_id)
    
    pending_doc = {
        "description": description,
        "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "fields": selected_fields
    }
    for field in selected_fields:
        main_window_instance.pendingDocuments.setdefault(field, []).append(pending_doc)
        update_pending_cell_background(main_window_instance, field)
    
    QMessageBox.information(main_window_instance, "Èxit",
                            f"Document pendent '{description}' afegit als camps: {', '.join(selected_fields)}.")
    listWidget.clear()
    pending_list = main_window_instance.pendingDocuments.get(cell_id, [])
    if pending_list:
        for doc in pending_list:
            listWidget.addItem(doc.get("description", "Sense descripció"))
    else:
        listWidget.addItem("No hi ha documents pendents")


def associate_pending_document(main_window_instance, doc_desc, cell_id, dialog):
    """
    Associa un document pendent (escollit per descripció) al fitxer seleccionat.
    Aquest document s'associa a totes les caselles d'introducció de text dels camps
    que s'havien seleccionat en el moment d'afegir-lo. Un cop associat, s'elimina el document pendent
    de TOTS els camps on apareix.
    """
    from PyQt5.QtWidgets import QFileDialog
    pending_list = main_window_instance.pendingDocuments.get(cell_id, [])
    target_doc = None
    for doc in pending_list:
        if doc.get("description", "Sense descripció") == doc_desc:
            target_doc = doc
            break
    if target_doc is None:
        dialog.accept()
        return
    filePath, _ = QFileDialog.getOpenFileName(main_window_instance, "Selecciona Document", "", "All Files (*)")
    if not filePath:
        return
    fileName = os.path.basename(filePath)
    target_doc["path"] = filePath
    target_doc["name"] = fileName
    target_doc["timestamp"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    fields = target_doc.get("fields", [])
    if not fields:
        fields = list(main_window_instance.selectedChanges)
    for field in fields:
        main_window_instance.associatedDocuments.setdefault(field, []).append(target_doc)
    for field in fields:
        if field in main_window_instance.pendingDocuments:
            try:
                main_window_instance.pendingDocuments[field].remove(target_doc)
            except ValueError:
                pass
        update_pending_cell_background(main_window_instance, field)
    from PyQt5.QtWidgets import QMessageBox
    QMessageBox.information(
        main_window_instance,
        "Èxit",
        f"Document '{fileName}' associat als camps: {', '.join(fields)} i eliminat de la llista."
    )
    dialog.accept()


In [5]:
# history_manager.py

import datetime
import os
from PyQt5.QtWidgets import (
    QDialog, QVBoxLayout, QListWidget, QListWidgetItem, QPushButton,
    QDialogButtonBox, QFileDialog, QMessageBox
)
from PyQt5.QtGui import QDesktopServices
from PyQt5.QtCore import Qt, QUrl

def show_history(main_window_instance, cell_id):
    dialog = QDialog(main_window_instance)
    dialog.setWindowTitle(f"Historial de canvis per al camp {cell_id}")
    layout = QVBoxLayout(dialog)
    listWidget = QListWidget()
    # Permet la selecció múltiple per afegir document a diversos canvis
    listWidget.setSelectionMode(QListWidget.MultiSelection)
    
    records = main_window_instance.history.get(cell_id, [])
    if records:
        n = len(records)
        # Afegim els registres en ordre invers (els més recents primer)
        for i, record in enumerate(records[::-1]):
            # Calcula l'índex original en la llista de registres
            record_index = n - 1 - i
            main_text = f"{i+1}. {record['date']}: {record['change']}"
            record_item = QListWidgetItem(main_text)
            # Emmagatzema l'índex original en el rol UserRole (per a poder-lo recuperar després)
            record_item.setData(Qt.UserRole, record_index)
            listWidget.addItem(record_item)
            # Afegim, si n'hi ha, un item per cada document associat (no seleccionable per afegir més documents)
            if record.get("documents"):
                for doc in record["documents"]:
                    doc_text = f"    Doc: {doc['name']}"
                    doc_item = QListWidgetItem(doc_text)
                    # Per aquests items, posem UserRole = None
                    doc_item.setData(Qt.UserRole, None)
                    listWidget.addItem(doc_item)
    else:
        listWidget.addItem("No hi ha canvis validats per aquest camp.")
    layout.addWidget(listWidget)
    
    btnAddDoc = QPushButton("Afegir document als canvis seleccionats")
    layout.addWidget(btnAddDoc)
    
    def onAdd():
        selected_items = listWidget.selectedItems()
        if not selected_items:
            QMessageBox.warning(dialog, "Selecció", "Selecciona almenys un canvi per afegir el document.")
            return
        records_list = main_window_instance.history.get(cell_id, [])
        filePath, _ = QFileDialog.getOpenFileName(
            main_window_instance,
            "Selecciona Document",
            "",
            "PDF Files (*.pdf);;Image Files (*.jpg *.png)"
        )
        if not filePath:
            return
        fileName = os.path.basename(filePath)
        doc = {
            "path": filePath,
            "name": fileName,
            "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        added = False
        for item in selected_items:
            record_index = item.data(Qt.UserRole)
            if record_index is None:
                continue  # No és un registre seleccionable
            # Comprovem que l'índex és vàlid
            if 0 <= record_index < len(records_list):
                records_list[record_index].setdefault("documents", []).append(doc)
                added = True
        if added:
            QMessageBox.information(main_window_instance, "Document Afegit",
                                    f"Document '{fileName}' afegit als canvis seleccionats.")
        else:
            QMessageBox.warning(dialog, "Selecció", "Cap canvi seleccionat és vàlid per afegir el document.")
        dialog.accept()
    btnAddDoc.clicked.connect(onAdd)
    
    def open_associated_document(item):
        file_path = item.data(Qt.UserRole)
        if file_path:
            QDesktopServices.openUrl(QUrl.fromLocalFile(file_path))
    listWidget.itemDoubleClicked.connect(open_associated_document)
    
    buttonBox = QDialogButtonBox(QDialogButtonBox.Ok)
    buttonBox.accepted.connect(dialog.accept)
    layout.addWidget(buttonBox)
    dialog.setLayout(layout)
    dialog.exec_()

def add_document_to_change(main_window_instance, cell_id, listWidget, dialog):
    records = main_window_instance.history.get(cell_id, [])
    if not records:
        QMessageBox.information(main_window_instance, "Sense canvis", "No hi ha canvis per aquest camp per afegir documents.")
        return
    selected_items = listWidget.selectedItems()
    if not selected_items:
        QMessageBox.warning(dialog, "Selecció", "Selecciona almenys un canvi per afegir el document.")
        return
    filePath, _ = QFileDialog.getOpenFileName(
        main_window_instance,
        "Selecciona Document",
        "",
        "PDF Files (*.pdf);;Image Files (*.jpg *.png)"
    )
    if not filePath:
        return
    fileName = os.path.basename(filePath)
    doc = {
        "path": filePath,
        "name": fileName,
        "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    records_list = records
    for item in selected_items:
        record_index = item.data(Qt.UserRole)
        if record_index is None:
            continue
        if 0 <= record_index < len(records_list):
            records_list[record_index].setdefault("documents", []).append(doc)
    QMessageBox.information(main_window_instance, "Document Afegit",
                            f"Document '{fileName}' afegit als canvis seleccionats.")
    dialog.accept()


In [6]:
# validation_manager.py

import datetime
from PyQt5.QtWidgets import QMessageBox

def validate_modifications(main_window_instance):
    # Només es validen els camps amb checkbox marcat.
    for group in main_window_instance.group_info:
        dv_widget = group["widget"]
        cell_id = f"{group['row']}-{group['col']}"
        if dv_widget.checkbox and dv_widget.checkbox.isChecked():
            if main_window_instance.pendingDocuments.get(cell_id):
                QMessageBox.warning(main_window_instance, "Error", "Documents pendents d'associar. No es pot validar")
                continue
            new_text = dv_widget.line_edit.text().strip()
            if new_text:
                old_text = dv_widget.label.text()
                changeRecord = {
                    "change": f"{old_text} → {new_text}",
                    "date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "documents": main_window_instance.associatedDocuments.get(cell_id, []).copy()
                }
                main_window_instance.history.setdefault(cell_id, []).append(changeRecord)
                dv_widget.update_official(new_text)
                main_window_instance.associatedDocuments[cell_id] = []

def toggle_changes_visibility(main_window_instance, toggle_button):
    # Comprova si els camps estan actualment ocults basant-se en un dels elements
    # (suposem que si el QLineEdit està ocult, també ho està el checkbox)
    currently_hidden = not main_window_instance.group_info[0]["widget"].line_edit.isVisible() if main_window_instance.group_info else True
    for group in main_window_instance.group_info:
        if currently_hidden:
            group["widget"].line_edit.show()
            group["widget"].checkbox.show()
        else:
            group["widget"].line_edit.hide()
            group["widget"].checkbox.hide()
    toggle_button.setText("Ocultar canvis" if currently_hidden else "Mostrar canvis")



In [7]:
# columns.py
# Aquest mòdul conté les definicions de les columnes predefinides

from PyQt5.QtWidgets import QDialog, QGridLayout, QCheckBox, QLabel, QHeaderView
from PyQt5.QtGui import QGuiApplication

COLUMNS_INICI = [
    "0.Percentatge de participació de la Generalitat", "5100_Particip.Denominació partícip (agregat)",
    "5100_Membres.Denominació membre", "5100_Membres.Percentatge drets de vot",
    "5100_Persones.Cognoms", "5100_Persones.Nom"
]

COLUMNS_DADES_ENTITAT = [
    "0.Codi Registre", "0.Marca o acrònim", "0.Tipus d'ens catàleg",
    "Naturalesa Jurídica (agregat)", "0.Naturalesa Jurídica (detall)", "0.NIF", "0.Departament d'adscripció",
    "Data d'efectes", "0.Règim jurídic singularitzat", "0.Grau de participació", "0.Via de participació",
    "Percentatge de participació de la Generalitat", "0.Mesura de participació", "0.Qualificador de participació",
    "Entitat independent", "0.Funcions, objecte social o finalitat",
    "Referència legal de les funcions, objecte social o finalitat", "0.Registre de fundacions",
    "Data de registre de fundacions", "0.Número del registre de fundacions", "0.Administració d'adscripció",
    "Situació registral", "0.Estat registral", "0.Data registral des de", "0.Data registral fins a",
    "Data d’efectes jurídics des de", "0.Data d’efectes jurídics fins a", "0.Tipus de via", "0.Nom de la via",
    "Número de la via", "0.Situació", "0.Comunitat", "0.Localitat", "0.Codi postal", "0.Província", "0.Telèfon",
    "WEB", "0.Adreça fitxa SAC", "0.E-mail", "0.Total % de participació Generalitat",
    "Total % de participació Generalitat (ponderat)", "0.Total % particip. Generalitat amb Universitats",
    "Total % particip. Generalitat amb Universitats (ponderat)", "0.Total % participació sector públic",
    "Mesura de la participació", "0.Observacions", "0.Òrgan de govern", "0.Nombre mínim de membres",
    "Nombre màxim de membres", "0.Total membres Generalitat", "0.Total membres sector públic",
    "Total membres (calculat)", "0.Total membres (informat)", "0.Total % vot Generalitat",
    "Total % vot sector públic", "0.Observacions_1"
]

COLUMNS_PARTICIPS = [
    "5100_Particip.Codi Catàleg", "5100_Particip.Denominació", "5100_Particip.Categorització partícip",
    "5100_Particip.Sectorització del partícip", "5100_Particip.Denominació partícip (agregat)",
    "5100_Particip.Codi Catàleg_1", "5100_Particip.P. Física", "5100_Particip.Vincle primari",
    "5100_Particip.Percentatge participació", "5100_Particip.Clàusules particulars",
    "5100_Particip.Percentatge del cap. Social (mercantils)", "5100_Particip.Vot de qualitat",
    "5100_Particip.Data alta partícip", "5100_Particip.Data baixa partícip"
]

COLUMNS_MEMBRES = [
    "5100_Membres.Codi Catàleg", "5100_Membres.Denominació", "5100_Membres.Denominació membre",
    "5100_Membres.P. Física", "5100_Membres.Membre partícip", "5100_Membres.Partícip o organisme",
    "5100_Membres.Codi Catàleg_1", "5100_Membres.Departament", "5100_Membres.Dret de vot",
    "5100_Membres.Categorització partícip", "5100_Membres.Sectorització del partícip",
    "5100_Membres.Nombre de membres", "5100_Membres.Percentatge drets de vot",
    "5100_Membres.Clàusules particulars", "5100_Membres.Vot de qualitat",
    "5100_Membres.Data alta membre", "5100_Membres.Data baixa membre"
]

COLUMNS_PERSONES = [
    "5100_Persones.Codi Catàleg", "5100_Persones.Denominació", "5100_Persones.Denominació membre",
    "5100_Persones.Qualificador de persona física, jurídica o vacant", "5100_Persones.DNI membre (persona física)",
    "5100_Persones.Cognoms", "5100_Persones.Nom", "5100_Persones.NIF membre (persona jurídica)",
    "5100_Persones.Denominació social", "5100_Persones.DNI persona representant",
    "5100_Persones.Cognoms persona representant", "5100_Persones.Nom persona representant",
    "5100_Persones.Càrrec en l'òrgan de govern superior", "5100_Persones.Data inici de vigència",
    "5100_Persones.Data final de vigència", "5100_Persones.Tipus de nomenament",
    "5100_Persones.Òrgan que el nomena", "5100_Persones.Lloc de treball o càrrec que exerceix"
]

from PyQt5.QtWidgets import QDialog, QGridLayout, QCheckBox, QLabel, QScrollArea, QWidget, QVBoxLayout
from PyQt5.QtGui import QGuiApplication

# ---------- DIÀLEG DE SELECCIÓ DE COLUMNES ----------
# Diàleg per a la selecció de columnes
class ColumnSelectionDialog(QDialog):
    def __init__(self, table, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Selecciona columnes visibles")
        self.table = table

        # Configurar l'àrea de desplaçament
        scroll_area = QScrollArea(self)
        scroll_area.setWidgetResizable(True)
        scroll_content = QWidget(scroll_area)
        scroll_area.setWidget(scroll_content)
        scroll_layout = QVBoxLayout(scroll_content)

        grid_layout = QGridLayout()
        scroll_layout.addLayout(grid_layout)

        self.setLayout(QVBoxLayout())
        self.layout().addWidget(scroll_area)
        self.checkboxes = []

        # Obtenim les dimensions de la pantalla
        screen_geometry = QGuiApplication.primaryScreen().availableGeometry()
        screen_width = screen_geometry.width()
        screen_height = screen_geometry.height()

        # Establim la mida de la finestra al 80% de les dimensions de la pantalla
        self.setFixedSize(int(screen_width * 0.8), int(screen_height * 0.8))

        # Ajustem l'espaiat del layout
        grid_layout.setHorizontalSpacing(5)
        grid_layout.setVerticalSpacing(5)

        # Definim les columnes fixes que sempre han d'estar visibles
        fixed_columns = ["0.Codi Catàleg", "0.Denominació"]

        n_cols = self.table.columnCount()

        # Agrupem les columnes per prefix
        columns_by_prefix = {}
        for i in range(n_cols):
            header_item = self.table.horizontalHeaderItem(i)
            if header_item is None:
                continue
            col_name = header_item.text().strip()
            prefix = col_name.split('.')[0] if '.' in col_name else col_name
            if prefix not in columns_by_prefix:
                columns_by_prefix[prefix] = []
            columns_by_prefix[prefix].append((i, col_name.split(".", 1)[1]))

        # Ordenem els prefixos i les columnes dins de cada prefix
        sorted_prefixes = sorted(columns_by_prefix.keys())
        for prefix in sorted_prefixes:
            columns_by_prefix[prefix].sort(key=lambda x: x[1])

        # Calculem max_per_col basat en l'alçada de la finestra i l'alçada de cada element de la interfície
        element_height = 20
        spacing = 5
        available_height = self.height() - 40  # Restem una mica per marges i espai extra
        max_per_col = (available_height // (element_height + spacing))-1

        # Afegim les columnes al diàleg segons el prefix
        col_idx = 0
        col_count = 0
        for prefix in sorted_prefixes:
            prefix_label = QLabel(f"Prefix: {prefix}")
            prefix_label.setFixedHeight(element_height)  # Ajusta l'alçada de l'etiqueta dels prefixos
            grid_layout.addWidget(prefix_label, 0, col_idx)
            row_idx = 1
            for i, col_name in columns_by_prefix[prefix]:
                cb = QCheckBox(col_name)
                cb.setFixedHeight(element_height)  # Ajusta l'alçada de les caselles de verificació
                if col_name in fixed_columns:
                    cb.setChecked(True)
                    cb.setEnabled(False)
                    # Forcem que la columna estigui visible
                    self.table.setColumnHidden(i, False)
                else:
                    cb.setChecked(not self.table.isColumnHidden(i))
                    cb.toggled.connect(lambda checked, col=i: self.table.setColumnHidden(col, not checked))
                grid_layout.addWidget(cb, row_idx, col_idx)
                self.checkboxes.append(cb)
                row_idx += 1
                col_count += 1
                if row_idx >= max_per_col:
                    row_idx = 1
                    col_idx += 1
            col_idx += 1

In [8]:
cadena = "5100_Persones.Prova.segona"
resultat = cadena.split(".", 1)[1]  # Agafa la part després del primer punt
print(resultat)  # Output: Prova


Prova.segona


# main_window.py

import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QTableWidget, QPushButton,
    QVBoxLayout, QHBoxLayout, QWidget, QLineEdit, QDialog, QListWidget, QDialogButtonBox
)
from PyQt5.QtCore import Qt, QUrl
from PyQt5.QtGui import QDesktopServices

# Suposem que els següents mòduls estan disponibles:
# from custom_widgets import DoubleViewWidget
# from excel_handler import load_excel_and_populate_table
# from pending_documents_manager import show_pending_documents
# from history_manager import show_history
# from validation_manager import validate_modifications
# from session_manager import save_session, load_session
# from columns import COLUMNS_INICI, COLUMNS_DADES_ENTITAT, COLUMNS_PARTICIPS, COLUMNS_MEMBRES, COLUMNS_PERSONES

def toggle_changes_visibility(main_window, button):
    if not hasattr(main_window, "changes_visible"):
        main_window.changes_visible = False
    main_window.changes_visible = not main_window.changes_visible
    for group in main_window.group_info:
        group["widget"].line_edit.setVisible(main_window.changes_visible)
        if hasattr(group["widget"], "checkbox"):
            group["widget"].checkbox.setVisible(main_window.changes_visible)
    if main_window.changes_visible:
        button.setText("Amaga canvis")
    else:
        button.setText("Mostra canvis")

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Gestor de Documents Pendents")
        
        # Inicialitza els diccionaris d'estat
        self.pendingDocuments = {}
        self.associatedDocuments = {}
        self.history = {}
        self.excel_df = None  # Aquí es guardarà el contingut original de l'Excel
        
        central_widget = QWidget()
        main_layout = QVBoxLayout(central_widget)
        
        # Primera fila: botons de selecció de columnes (una sola fila)
        top_button_layout = QHBoxLayout()
        self.select_columns_button = QPushButton("Seleccionar columnes visibles")
        self.select_columns_button.clicked.connect(self.open_column_selection_dialog)
        top_button_layout.addWidget(self.select_columns_button)
        
        self.predefined_buttons = {}
        self.btn_inici = QPushButton("Inici")
        self.btn_inici.setCheckable(True)
        top_button_layout.addWidget(self.btn_inici)
        self.predefined_buttons[frozenset(COLUMNS_INICI)] = self.btn_inici
        
        self.btn_dades_entitat = QPushButton("Dades d'entitat")
        self.btn_dades_entitat.setCheckable(True)
        top_button_layout.addWidget(self.btn_dades_entitat)
        self.predefined_buttons[frozenset(COLUMNS_DADES_ENTITAT)] = self.btn_dades_entitat
        
        self.btn_particip = QPushButton("Partícips")
        self.btn_particip.setCheckable(True)
        top_button_layout.addWidget(self.btn_particip)
        self.predefined_buttons[frozenset(COLUMNS_PARTICIPS)] = self.btn_particip
        
        self.btn_membres = QPushButton("Membres")
        self.btn_membres.setCheckable(True)
        top_button_layout.addWidget(self.btn_membres)
        self.predefined_buttons[frozenset(COLUMNS_MEMBRES)] = self.btn_membres
        
        self.btn_persones = QPushButton("Persones")
        self.btn_persones.setCheckable(True)
        top_button_layout.addWidget(self.btn_persones)
        self.predefined_buttons[frozenset(COLUMNS_PERSONES)] = self.btn_persones
        
        for btn in self.predefined_buttons.values():
            btn.toggled.connect(self.update_predefined_columns_visibility)
        
        main_layout.addLayout(top_button_layout)
        
        # Fila de filtres: s'afegeix un QLineEdit per a cada columna a la primera fila de la taula
        self.filter_edits = []
        
        # Segona fila: botons de mostrar/amagar modificacions i validar, a l'esquerra
        bottom_button_layout = QHBoxLayout()
        bottom_button_layout.setAlignment(Qt.AlignLeft)
        self.validate_button = QPushButton("Validar modificacions")
        self.validate_button.clicked.connect(lambda: validate_modifications(self))
        bottom_button_layout.addWidget(self.validate_button)
        
        self.toggle_button = QPushButton("Mostrar / Ocultar canvis")
        self.toggle_button.clicked.connect(lambda: toggle_changes_visibility(self, self.toggle_button))
        bottom_button_layout.addWidget(self.toggle_button)
        
        main_layout.addLayout(bottom_button_layout)
        
        # Taula per mostrar les dades
        self.table = QTableWidget()
        main_layout.addWidget(self.table)
        
        self.setCentralWidget(central_widget)
        
        self.table.setContextMenuPolicy(Qt.CustomContextMenu)
        self.table.customContextMenuRequested.connect(self.onContextMenu)
        self.table.horizontalHeader().setStyleSheet("font-weight: bold; background-color: lightgrey;")
        
        self.group_info = []
        self.pendingDocuments = {}
        self.associatedDocuments = {}
        self.history = {}
        self.selectedChanges = set()
        self.changes_visible = False

        # Carrega l'Excel i omple la taula amb els widgets corresponents.
        load_excel_and_populate_table(self.table, self.group_info, self)
        # Afegeix la fila de filtres; aquesta fila s'insereix com la fila 0
        self.add_filter_row()
        
        # Amaga les caselles de modificació i també les de selecció (els widgets de les dades, no els filtres):
        for group in self.group_info:
            group["widget"].line_edit.hide()
            if hasattr(group["widget"], "checkbox"):
                group["widget"].checkbox.hide()

        self.update_visible_rows()
    
    def add_filter_row(self):
        """Insereix una fila de filtres a l'índex 0 de la taula, sota les capçaleres."""
        self.table.insertRow(0)
        self.filter_edits = []
        num_cols = self.table.columnCount()
        for col in range(num_cols):
            header_item = self.table.horizontalHeaderItem(col)
            header_text = header_item.text().strip() if header_item else f"Col {col+1}"
            edit = QLineEdit()
            edit.setPlaceholderText(f"Filtrar {header_text}")
            edit.textChanged.connect(self.apply_filters)
            self.table.setCellWidget(0, col, edit)
            self.filter_edits.append(edit)
    
    def apply_filters(self):
        """
        Aplica els filtres de la fila 0 a les files de dades (a partir de la fila 1).
        Per cada cel·la de cada fila visible, es combina el text del widget (si n'hi ha) 
        (tant el de l'etiqueta com el de la QLineEdit) o el text de l'item, i si el filtre no
        es troba en aquest text, la fila es fa invisible.
        """
        num_rows = self.table.rowCount()
        num_cols = self.table.columnCount()
        for row in range(1, num_rows):  # Comença per la fila 1 (les dades)
            show_row = True
            for col in range(num_cols):
                if not self.table.isColumnHidden(col):
                    filter_text = self.filter_edits[col].text().strip().lower()
                    if filter_text:
                        cell_text = ""
                        widget = self.table.cellWidget(row, col)
                        if widget is not None:
                            texts = []
                            if hasattr(widget, "label"):
                                texts.append(widget.label.text().strip().lower())
                            if hasattr(widget, "line_edit"):
                                texts.append(widget.line_edit.text().strip().lower())
                            cell_text = " ".join(texts)
                        else:
                            item = self.table.item(row, col)
                            if item is not None:
                                cell_text = item.text().strip().lower()
                        if filter_text not in cell_text:
                            show_row = False
                            break
            self.table.setRowHidden(row, not show_row)
    
    def open_column_selection_dialog(self):
        dialog = ColumnSelectionDialog(self.table, self)
        dialog.exec_()
        self.update_visible_rows()
        
    def update_predefined_columns_visibility(self):
        fixed_columns = ["0.Codi Catàleg", "0.Denominació"]
        active_predefined = set()
        for columns_set, btn in self.predefined_buttons.items():
            if btn.isChecked():
                active_predefined.update(columns_set)
        for col in range(self.table.columnCount()):
            header = self.table.horizontalHeaderItem(col).text().strip()
            if header in fixed_columns:
                self.table.setColumnHidden(col, False)
            else:
                visible = header in active_predefined if active_predefined else True
                self.table.setColumnHidden(col, not visible)
        self.update_visible_rows()
    
    def update_visible_rows(self):
        factor = 10
        num_columnes_visibles = sum(not self.table.isColumnHidden(col) for col in range(self.table.columnCount()))
        max_files = num_columnes_visibles * factor
        for row in range(1, self.table.rowCount()):  # Exclou la fila 0 (els filtres)
            self.table.setRowHidden(row, row >= max_files)
        self.apply_filters()

    def onSelectionChanged(self, cell_id, selected):
        if selected:
            self.selectedChanges.add(cell_id)
        else:
            self.selectedChanges.discard(cell_id)
    
    def onContextMenu(self, pos):
        index = self.table.indexAt(pos)
        # Si es fa clic a la fila de filtres, no es mostra cap menú
        if index.row() == 0:
            return
        if not index.isValid():
            return
        row = index.row()
        if index.column() % 2 == 0:
            cell_id = f"{row}-{index.column() // 2}"
        else:
            cell_id = f"{row}-{(index.column()-1) // 2}"
        from PyQt5.QtWidgets import QMenu
        menu = QMenu(self)
        actionPending = menu.addAction("Documents pendents")
        action = menu.exec_(self.table.viewport().mapToGlobal(pos))
        if action == actionPending:
            show_pending_documents(self, cell_id)
    
    def setCellBackground(self, cell_id, color):
        try:
            row, col = map(int, cell_id.split('-'))
        except Exception:
            return
        from PyQt5.QtGui import QBrush
        item = self.table.item(row, col * 2)
        if item:
            item.setBackground(QBrush(color))
    
    def associateDocumentFromLineEdit(self, cell_id):
        from PyQt5.QtWidgets import QFileDialog, QMessageBox
        import os, datetime
        filePath, _ = QFileDialog.getOpenFileName(self, "Selecciona Document", "")
        if filePath:
            fileName = os.path.basename(filePath)
            doc = {
                "path": filePath,
                "name": fileName,
                "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            self.associatedDocuments.setdefault(cell_id, []).append(doc)
            QMessageBox.information(self, "Document Associat", f"Document '{fileName}' associat correctament al camp {cell_id}.")
    
    def showPendingDocumentsFromLineEdit(self, cell_id):
        show_pending_documents(self, cell_id)
    
    def showHistoryFromLabel(self, cell_id):
        show_history(self, cell_id)
    
    def showAssociatedDocumentsFromLineEdit(self, cell_id):
        dialog = QDialog(self)
        dialog.setWindowTitle(f"Documents associats per al camp {cell_id}")
        layout = QVBoxLayout(dialog)
        listWidget = QListWidget()
        associated = self.associatedDocuments.get(cell_id, [])
        if associated:
            for doc in associated:
                item = QListWidgetItem(f"{doc.get('name','No nom')} - {doc.get('timestamp','')}")
                item.setData(Qt.UserRole, doc.get('path'))
                listWidget.addItem(item)
        else:
            listWidget.addItem("No hi ha documents associats")
        layout.addWidget(listWidget)
        buttonBox = QDialogButtonBox(QDialogButtonBox.Ok)
        buttonBox.accepted.connect(dialog.accept)
        layout.addWidget(buttonBox)
        listWidget.itemClicked.connect(self.openAssociatedDocument)
        dialog.setLayout(layout)
        dialog.exec_()

    def openAssociatedDocument(self, item):
        file_path = item.data(Qt.UserRole)
        if file_path:
            QDesktopServices.openUrl(QUrl.fromLocalFile(file_path))

    def refresh_table(self):
        for group in self.group_info:
            cell_id = group["widget"].cell_id
            if cell_id in self.associatedDocuments and self.associatedDocuments[cell_id]:
                group["widget"].label.setStyleSheet("background-color: #ccffcc;")
            else:
                group["widget"].label.setStyleSheet("")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

# main_window.py

import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QTableWidget, QPushButton,
    QVBoxLayout, QHBoxLayout, QWidget, QLineEdit, QDialog, QListWidget, QDialogButtonBox, QListWidgetItem
)
from PyQt5.QtCore import Qt, QUrl
from PyQt5.QtGui import QDesktopServices

# Suposem que els següents mòduls estan disponibles:
# from custom_widgets import DoubleViewWidget
# from excel_handler import load_excel_and_populate_table
# from pending_documents_manager import show_pending_documents
# from history_manager import show_history
# from validation_manager import validate_modifications
# from session_manager import save_session, load_session
# from columns import COLUMNS_INICI, COLUMNS_DADES_ENTITAT, COLUMNS_PARTICIPS, COLUMNS_MEMBRES, COLUMNS_PERSONES

def toggle_changes_visibility(main_window, button):
    if not hasattr(main_window, "changes_visible"):
        main_window.changes_visible = False
    main_window.changes_visible = not main_window.changes_visible
    for group in main_window.group_info:
        group["widget"].line_edit.setVisible(main_window.changes_visible)
        if hasattr(group["widget"], "checkbox"):
            group["widget"].checkbox.setVisible(main_window.changes_visible)
    if main_window.changes_visible:
        button.setText("Amaga canvis")
    else:
        button.setText("Mostra canvis")

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Gestor de Documents Pendents")
        
        # Inicialitza els diccionaris d'estat
        self.pendingDocuments = {}
        self.associatedDocuments = {}
        self.history = {}
        self.excel_df = None  # Aquí es guardarà el contingut original de l'Excel
        
        central_widget = QWidget()
        main_layout = QVBoxLayout(central_widget)
        
        # Primera fila: botons de selecció de columnes (una sola fila)
        top_button_layout = QHBoxLayout()
        self.select_columns_button = QPushButton("Seleccionar columnes visibles")
        self.select_columns_button.clicked.connect(self.open_column_selection_dialog)
        top_button_layout.addWidget(self.select_columns_button)
        
        self.predefined_buttons = {}
        self.btn_inici = QPushButton("Inici")
        self.btn_inici.setCheckable(True)
        top_button_layout.addWidget(self.btn_inici)
        self.predefined_buttons[frozenset(COLUMNS_INICI)] = self.btn_inici
        
        self.btn_dades_entitat = QPushButton("Dades d'entitat")
        self.btn_dades_entitat.setCheckable(True)
        top_button_layout.addWidget(self.btn_dades_entitat)
        self.predefined_buttons[frozenset(COLUMNS_DADES_ENTITAT)] = self.btn_dades_entitat
        
        self.btn_particip = QPushButton("Partícips")
        self.btn_particip.setCheckable(True)
        top_button_layout.addWidget(self.btn_particip)
        self.predefined_buttons[frozenset(COLUMNS_PARTICIPS)] = self.btn_particip
        
        self.btn_membres = QPushButton("Membres")
        self.btn_membres.setCheckable(True)
        top_button_layout.addWidget(self.btn_membres)
        self.predefined_buttons[frozenset(COLUMNS_MEMBRES)] = self.btn_membres
        
        self.btn_persones = QPushButton("Persones")
        self.btn_persones.setCheckable(True)
        top_button_layout.addWidget(self.btn_persones)
        self.predefined_buttons[frozenset(COLUMNS_PERSONES)] = self.btn_persones
        
        for btn in self.predefined_buttons.values():
            btn.toggled.connect(self.update_predefined_columns_visibility)
        
        main_layout.addLayout(top_button_layout)
        
        # Fila de filtres: s'afegeix un QLineEdit per a cada columna a la primera fila de la taula
        self.filter_edits = []
        
        # Segona fila: botons de mostrar/amagar modificacions i validar, a l'esquerra
        bottom_button_layout = QHBoxLayout()
        bottom_button_layout.setAlignment(Qt.AlignLeft)
        self.validate_button = QPushButton("Validar modificacions")
        self.validate_button.clicked.connect(lambda: validate_modifications(self))
        bottom_button_layout.addWidget(self.validate_button)
        
        self.toggle_button = QPushButton("Mostrar / Ocultar canvis")
        self.toggle_button.clicked.connect(lambda: toggle_changes_visibility(self, self.toggle_button))
        bottom_button_layout.addWidget(self.toggle_button)
        
        main_layout.addLayout(bottom_button_layout)
        
        # Taula per mostrar les dades
        self.table = QTableWidget()
        main_layout.addWidget(self.table)
        
        self.setCentralWidget(central_widget)
        
        self.table.setContextMenuPolicy(Qt.CustomContextMenu)
        self.table.customContextMenuRequested.connect(self.onContextMenu)
        self.table.horizontalHeader().setStyleSheet("font-weight: bold; background-color: lightgrey;")
        
        self.group_info = []
        self.pendingDocuments = {}
        self.associatedDocuments = {}
        self.history = {}
        self.selectedChanges = set()
        self.changes_visible = False

        # Carrega l'Excel i omple la taula amb els widgets corresponents.
        load_excel_and_populate_table(self.table, self.group_info, self)
        # Afegeix la fila de filtres; aquesta fila s'insereix com la fila 0
        self.add_filter_row()
        
        # Amaga les caselles de modificació i també les de selecció (els widgets de les dades, no els filtres):
        for group in self.group_info:
            group["widget"].line_edit.hide()
            if hasattr(group["widget"], "checkbox"):
                group["widget"].checkbox.hide()

        self.update_visible_rows()
    
    def add_filter_row(self):
        """Insereix una fila de filtres a l'índex 0 de la taula, sota les capçaleres."""
        self.table.insertRow(0)
        self.filter_edits = []
        num_cols = self.table.columnCount()
        for col in range(num_cols):
            header_item = self.table.horizontalHeaderItem(col)
            header_text = header_item.text().strip() if header_item else f"Col {col+1}"
            edit = QLineEdit()
            edit.setPlaceholderText(f"Filtrar {header_text}")
            edit.textChanged.connect(self.apply_filters)
            self.table.setCellWidget(0, col, edit)
            self.filter_edits.append(edit)
    
    def apply_filters(self):
        """
        Aplica els filtres de la fila 0 a les files de dades (a partir de la fila 1).
        Per cada cel·la de cada fila visible, es combina el text del widget (si n'hi ha) 
        (tant el de l'etiqueta com el de la QLineEdit) o el text de l'item, i si el filtre no
        es troba en aquest text, la fila es fa invisible.
        """
        num_rows = self.table.rowCount()
        num_cols = self.table.columnCount()
        for row in range(1, num_rows):  # Comença per la fila 1 (les dades)
            show_row = True
            for col in range(num_cols):
                if not self.table.isColumnHidden(col):
                    filter_text = self.filter_edits[col].text().strip().lower()
                    if filter_text:
                        cell_text = ""
                        widget = self.table.cellWidget(row, col)
                        if widget is not None:
                            texts = []
                            if hasattr(widget, "label"):
                                texts.append(widget.label.text().strip().lower())
                            if hasattr(widget, "line_edit"):
                                texts.append(widget.line_edit.text().strip().lower())
                            cell_text = " ".join(texts)
                        else:
                            item = self.table.item(row, col)
                            if item is not None:
                                cell_text = item.text().strip().lower()
                        if filter_text not in cell_text:
                            show_row = False
                            break
            self.table.setRowHidden(row, not show_row)
    
    def open_column_selection_dialog(self):
        dialog = ColumnSelectionDialog(self.table, self)
        dialog.exec_()
        self.update_visible_rows()
        
    def update_predefined_columns_visibility(self):
        fixed_columns = ["0.Codi Catàleg", "0.Denominació"]
        active_predefined = set()
        for columns_set, btn in self.predefined_buttons.items():
            if btn.isChecked():
                active_predefined.update(columns_set)
        for col in range(self.table.columnCount()):
            header = self.table.horizontalHeaderItem(col).text().strip()
            if header in fixed_columns:
                self.table.setColumnHidden(col, False)
            else:
                visible = header in active_predefined if active_predefined else True
                self.table.setColumnHidden(col, not visible)
        self.update_visible_rows()
    
    def update_visible_rows(self):
        factor = 10
        num_columnes_visibles = sum(not self.table.isColumnHidden(col) for col in range(self.table.columnCount()))
        max_files = num_columnes_visibles * factor
        for row in range(1, self.table.rowCount()):  # Exclou la fila 0 (els filtres)
            self.table.setRowHidden(row, row >= max_files)
        self.apply_filters()

    def onSelectionChanged(self, cell_id, selected):
        if selected:
            self.selectedChanges.add(cell_id)
        else:
            self.selectedChanges.discard(cell_id)
    
    def onContextMenu(self, pos):
        index = self.table.indexAt(pos)
        # Si es fa clic a la fila de filtres, no es mostra cap menú
        if index.row() == 0:
            return
        if not index.isValid():
            return
        row = index.row()
        if index.column() % 2 == 0:
            cell_id = f"{row}-{index.column() // 2}"
        else:
            cell_id = f"{row}-{(index.column()-1) // 2}"
        from PyQt5.QtWidgets import QMenu
        menu = QMenu(self)
        actionPending = menu.addAction("Documents pendents")
        action = menu.exec_(self.table.viewport().mapToGlobal(pos))
        if action == actionPending:
            show_pending_documents(self, cell_id)
    
    def setCellBackground(self, cell_id, color):
        try:
            row, col = map(int, cell_id.split('-'))
        except Exception:
            return
        from PyQt5.QtGui import QBrush
        item = self.table.item(row, col * 2)
        if item:
            item.setBackground(QBrush(color))
    
    def associateDocumentFromLineEdit(self, cell_id):
        from PyQt5.QtWidgets import QFileDialog, QMessageBox
        import os, datetime
        filePath, _ = QFileDialog.getOpenFileName(self, "Selecciona Document", "")
        if filePath:
            fileName = os.path.basename(filePath)
            doc = {
                "path": filePath,
                "name": fileName,
                "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            self.associatedDocuments.setdefault(cell_id, []).append(doc)
            QMessageBox.information(self, "Document Associat", f"Document '{fileName}' associat correctament al camp {cell_id}.")
    
    def showPendingDocumentsFromLineEdit(self, cell_id):
        show_pending_documents(self, cell_id)
    
    def showHistoryFromLabel(self, cell_id):
        dialog = QDialog(self)
        dialog.setWindowTitle(f"Historial de canvis per al camp {cell_id}")
        layout = QVBoxLayout(dialog)
        listWidget = QListWidget()
        records = self.history.get(cell_id, [])
        if records:
            for record in records:
                item = QListWidgetItem(f"{record['date']}: {record['change']}")
                listWidget.addItem(item)
                if 'documents' in record:
                    for doc in record['documents']:
                        doc_item = QListWidgetItem(f"  Document: {doc['name']}")
                        doc_item.setData(Qt.UserRole, doc['path'])
                        listWidget.addItem(doc_item)
        else:
            listWidget.addItem("No hi ha canvis per aquest camp")
        layout.addWidget(listWidget)
        buttonBox = QDialogButtonBox(QDialogButtonBox.Ok)
        buttonBox.accepted.connect(dialog.accept)
        layout.addWidget(buttonBox)
        listWidget.itemClicked.connect(self.openAssociatedDocument)
        dialog.setLayout(layout)
        dialog.exec_()

    def openAssociatedDocument(self, item):
        file_path = item.data(Qt.UserRole)
        if file_path:
            QDesktopServices.openUrl(QUrl.fromLocalFile(file_path))

    def showAssociatedDocumentsFromLineEdit(self, cell_id):
        dialog = QDialog(self)
        dialog.setWindowTitle(f"Documents associats per al camp {cell_id}")
        layout = QVBoxLayout(dialog)
        listWidget = QListWidget()
        associated = self.associatedDocuments.get(cell_id, [])
        if associated:
            for doc in associated:
                item = QListWidgetItem(f"{doc.get('name','No nom')} - {doc.get('timestamp','')}")
                item.setData(Qt.UserRole, doc.get('path'))
                listWidget.addItem(item)
        else:
            listWidget.addItem("No hi ha documents associats")
        layout.addWidget(listWidget)
        buttonBox = QDialogButtonBox(QDialogButtonBox.Ok)
        buttonBox.accepted.connect(dialog.accept)
        layout.addWidget(buttonBox)
        listWidget.itemClicked.connect(self.openAssociatedDocument)
        dialog.setLayout(layout)
        dialog.exec_()

    def refresh_table(self):
        for group in self.group_info:
            cell_id = group["widget"].cell_id
            if cell_id in self.associatedDocuments and self.associatedDocuments[cell_id]:
                group["widget"].label.setStyleSheet("background-color: #ccffcc;")
            else:
                group["widget"].label.setStyleSheet("")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

In [9]:
# main_window.py

import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QTableWidget, QPushButton,
    QVBoxLayout, QHBoxLayout, QWidget, QLineEdit, QDialog, QListWidget, QDialogButtonBox, QListWidgetItem, QMessageBox
)
from PyQt5.QtCore import Qt, QUrl
from PyQt5.QtGui import QDesktopServices
import datetime

# Suposem que els següents mòduls estan disponibles:
# from custom_widgets import DoubleViewWidget
# from excel_handler import load_excel_and_populate_table
# from pending_documents_manager import show_pending_documents
# from history_manager import show_history
# from validation_manager import validate_modifications
# from session_manager import save_session, load_session
# from columns import COLUMNS_INICI, COLUMNS_DADES_ENTITAT, COLUMNS_PARTICIPS, COLUMNS_MEMBRES, COLUMNS_PERSONES

def toggle_changes_visibility(main_window, button):
    if not hasattr(main_window, "changes_visible"):
        main_window.changes_visible = False
    main_window.changes_visible = not main_window.changes_visible
    for group in main_window.group_info:
        group["widget"].line_edit.setVisible(main_window.changes_visible)
        if hasattr(group["widget"], "checkbox"):
            group["widget"].checkbox.setVisible(main_window.changes_visible)
    if main_window.changes_visible:
        button.setText("Amaga canvis")
    else:
        button.setText("Mostra canvis")

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Gestor de Documents Pendents")
        
        # Inicialitza els diccionaris d'estat
        self.pendingDocuments = {}
        self.associatedDocuments = {}
        self.history = {}
        self.excel_df = None  # Aquí es guardarà el contingut original de l'Excel
        
        central_widget = QWidget()
        main_layout = QVBoxLayout(central_widget)
        
        # Primera fila: botons de selecció de columnes (una sola fila)
        top_button_layout = QHBoxLayout()
        self.select_columns_button = QPushButton("Seleccionar columnes visibles")
        self.select_columns_button.clicked.connect(self.open_column_selection_dialog)
        top_button_layout.addWidget(self.select_columns_button)
        
        self.predefined_buttons = {}
        self.btn_inici = QPushButton("Inici")
        self.btn_inici.setCheckable(True)
        top_button_layout.addWidget(self.btn_inici)
        self.predefined_buttons[frozenset(COLUMNS_INICI)] = self.btn_inici
        
        self.btn_dades_entitat = QPushButton("Dades d'entitat")
        self.btn_dades_entitat.setCheckable(True)
        top_button_layout.addWidget(self.btn_dades_entitat)
        self.predefined_buttons[frozenset(COLUMNS_DADES_ENTITAT)] = self.btn_dades_entitat
        
        self.btn_particip = QPushButton("Partícips")
        self.btn_particip.setCheckable(True)
        top_button_layout.addWidget(self.btn_particip)
        self.predefined_buttons[frozenset(COLUMNS_PARTICIPS)] = self.btn_particip
        
        self.btn_membres = QPushButton("Membres")
        self.btn_membres.setCheckable(True)
        top_button_layout.addWidget(self.btn_membres)
        self.predefined_buttons[frozenset(COLUMNS_MEMBRES)] = self.btn_membres
        
        self.btn_persones = QPushButton("Persones")
        self.btn_persones.setCheckable(True)
        top_button_layout.addWidget(self.btn_persones)
        self.predefined_buttons[frozenset(COLUMNS_PERSONES)] = self.btn_persones
        
        for btn in self.predefined_buttons.values():
            btn.toggled.connect(self.update_predefined_columns_visibility)
        
        main_layout.addLayout(top_button_layout)
        
        # Fila de filtres: s'afegeix un QLineEdit per a cada columna a la primera fila de la taula
        self.filter_edits = []
        
        # Segona fila: botons de mostrar/amagar modificacions i validar, a l'esquerra
        bottom_button_layout = QHBoxLayout()
        bottom_button_layout.setAlignment(Qt.AlignLeft)
        self.validate_button = QPushButton("Validar modificacions")
        self.validate_button.clicked.connect(lambda: validate_modifications(self))
        bottom_button_layout.addWidget(self.validate_button)
        
        self.toggle_button = QPushButton("Mostrar / Ocultar canvis")
        self.toggle_button.clicked.connect(lambda: toggle_changes_visibility(self, self.toggle_button))
        bottom_button_layout.addWidget(self.toggle_button)
        
        main_layout.addLayout(bottom_button_layout)
        
        # Taula per mostrar les dades
        self.table = QTableWidget()
        main_layout.addWidget(self.table)
        
        self.setCentralWidget(central_widget)
        
        self.table.setContextMenuPolicy(Qt.CustomContextMenu)
        self.table.customContextMenuRequested.connect(self.onContextMenu)
        self.table.horizontalHeader().setStyleSheet("font-weight: bold; background-color: lightgrey;")
        
        self.group_info = []
        self.pendingDocuments = {}
        self.associatedDocuments = {}
        self.history = {}
        self.selectedChanges = set()
        self.changes_visible = False

        # Carrega l'Excel i omple la taula amb els widgets corresponents.
        load_excel_and_populate_table(self.table, self.group_info, self)
        # Afegeix la fila de filtres; aquesta fila s'insereix com la fila 0
        self.add_filter_row()
        
        # Amaga les caselles de modificació i també les de selecció (els widgets de les dades, no els filtres):
        for group in self.group_info:
            group["widget"].line_edit.hide()
            if hasattr(group["widget"], "checkbox"):
                group["widget"].checkbox.hide()

        self.update_visible_rows()
    
    def add_filter_row(self):
        """Insereix una fila de filtres a l'índex 0 de la taula, sota les capçaleres."""
        self.table.insertRow(0)
        self.filter_edits = []
        num_cols = self.table.columnCount()
        for col in range(num_cols):
            header_item = self.table.horizontalHeaderItem(col)
            header_text = header_item.text().strip() if header_item else f"Col {col+1}"
            edit = QLineEdit()
            edit.setPlaceholderText(f"Filtrar {header_text}")
            edit.textChanged.connect(self.apply_filters)
            self.table.setCellWidget(0, col, edit)
            self.filter_edits.append(edit)
    
    def apply_filters(self):
        """
        Aplica els filtres de la fila 0 a les files de dades (a partir de la fila 1).
        Per cada cel·la de cada fila visible, es combina el text del widget (si n'hi ha) 
        (tant el de l'etiqueta com el de la QLineEdit) o el text de l'item, i si el filtre no
        es troba en aquest text, la fila es fa invisible.
        """
        num_rows = self.table.rowCount()
        num_cols = self.table.columnCount()
        for row in range(1, num_rows):  # Comença per la fila 1 (les dades)
            show_row = True
            for col in range(num_cols):
                if not self.table.isColumnHidden(col):
                    filter_text = self.filter_edits[col].text().strip().lower()
                    if filter_text:
                        cell_text = ""
                        widget = self.table.cellWidget(row, col)
                        if widget is not None:
                            texts = []
                            if hasattr(widget, "label"):
                                texts.append(widget.label.text().strip().lower())
                            if hasattr(widget, "line_edit"):
                                texts.append(widget.line_edit.text().strip().lower())
                            cell_text = " ".join(texts)
                        else:
                            item = self.table.item(row, col)
                            if item is not None:
                                cell_text = item.text().strip().lower()
                        if filter_text not in cell_text:
                            show_row = False
                            break
            self.table.setRowHidden(row, not show_row)
    
    def open_column_selection_dialog(self):
        dialog = ColumnSelectionDialog(self.table, self)
        dialog.exec_()
        self.update_visible_rows()
        
    def update_predefined_columns_visibility(self):
        fixed_columns = ["0.Codi Catàleg", "0.Denominació"]
        active_predefined = set()
        for columns_set, btn in self.predefined_buttons.items():
            if btn.isChecked():
                active_predefined.update(columns_set)
        for col in range(self.table.columnCount()):
            header = self.table.horizontalHeaderItem(col).text().strip()
            if header in fixed_columns:
                self.table.setColumnHidden(col, False)
            else:
                visible = header in active_predefined if active_predefined else True
                self.table.setColumnHidden(col, not visible)
        self.update_visible_rows()
    
    def update_visible_rows(self):
        factor = 10
        num_columnes_visibles = sum(not self.table.isColumnHidden(col) for col in range(self.table.columnCount()))
        max_files = num_columnes_visibles * factor
        for row in range(1, self.table.rowCount()):  # Exclou la fila 0 (els filtres)
            self.table.setRowHidden(row, row >= max_files)
        self.apply_filters()

    def onSelectionChanged(self, cell_id, selected):
        if selected:
            self.selectedChanges.add(cell_id)
        else:
            self.selectedChanges.discard(cell_id)
    
    def onContextMenu(self, pos):
        index = self.table.indexAt(pos)
        # Si es fa clic a la fila de filtres, no es mostra cap menú
        if index.row() == 0:
            return
        if not index.isValid():
            return
        row = index.row()
        if index.column() % 2 == 0:
            cell_id = f"{row}-{index.column() // 2}"
        else:
            cell_id = f"{row}-{(index.column()-1) // 2}"
        from PyQt5.QtWidgets import QMenu
        menu = QMenu(self)
        actionPending = menu.addAction("Documents pendents")
        action = menu.exec_(self.table.viewport().mapToGlobal(pos))
        if action == actionPending:
            show_pending_documents(self, cell_id)
    
    def setCellBackground(self, cell_id, color):
        try:
            row, col = map(int, cell_id.split('-'))
        except Exception:
            return
        from PyQt5.QtGui import QBrush
        item = self.table.item(row, col * 2)
        if item:
            item.setBackground(QBrush(color))
    
    def associateDocumentFromLineEdit(self, cell_id):
        from PyQt5.QtWidgets import QFileDialog, QMessageBox
        import os, datetime
        filePath, _ = QFileDialog.getOpenFileName(self, "Selecciona Document", "")
        if filePath:
            fileName = os.path.basename(filePath)
            doc = {
                "path": filePath,
                "name": fileName,
                "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            self.associatedDocuments.setdefault(cell_id, []).append(doc)
            QMessageBox.information(self, "Document Associat", f"Document '{fileName}' associat correctament al camp {cell_id}.")
    
    def showPendingDocumentsFromLineEdit(self, cell_id):
        show_pending_documents(self, cell_id)
    
    def showHistoryFromLabel(self, cell_id):
        dialog = QDialog(self)
        dialog.setWindowTitle(f"Historial de canvis per al camp {cell_id}")
        layout = QVBoxLayout(dialog)
        listWidget = QListWidget()
        records = self.history.get(cell_id, [])
        if records:
            for record in records:
                item = QListWidgetItem(f"{record['date']}: {record['change']}")
                item.setData(Qt.UserRole, record)
                listWidget.addItem(item)
                if 'documents' in record:
                    for doc in record['documents']:
                        doc_item = QListWidgetItem(f"  Document: {doc['name']}")
                        doc_item.setData(Qt.UserRole, doc['path'])
                        listWidget.addItem(doc_item)
        else:
            listWidget.addItem("No hi ha canvis per aquest camp")
        layout.addWidget(listWidget)
        
        restore_button = QPushButton("Restaurar canvi")
        restore_button.clicked.connect(lambda: self.restore_change(cell_id, listWidget))
        layout.addWidget(restore_button)
        
        buttonBox = QDialogButtonBox(QDialogButtonBox.Ok)
        buttonBox.accepted.connect(dialog.accept)
        layout.addWidget(buttonBox)
        
        listWidget.itemClicked.connect(self.openAssociatedDocument)
        dialog.setLayout(layout)
        dialog.exec_()

    def restore_change(self, cell_id, listWidget):
        selected_items = listWidget.selectedItems()
        if not selected_items:
            QMessageBox.warning(self, "Selecció", "Selecciona un canvi per restaurar.")
            return
        selected_item = selected_items[0]
        record = selected_item.data(Qt.UserRole)
        if not record:
            QMessageBox.warning(self, "Selecció", "Selecciona un canvi vàlid per restaurar.")
            return

        # Obtenir l'últim text abans de la restauració
        current_text = None
        for group in self.group_info:
            if group["widget"].cell_id == cell_id:
                current_text = group["widget"].label.text()
                break
        if current_text is None:
            QMessageBox.warning(self, "Error", "No s'ha pogut trobar el text actual.")
            return
        
        new_text = record['change'].split("→")[-1].strip()
        new_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # Actualitza el text i els documents associats
        for group in self.group_info:
            if group["widget"].cell_id == cell_id:
                group["widget"].label.setText(new_text)
                group["widget"].line_edit.setText(new_text)
                group["widget"].label.setStyleSheet("background-color: #ccffcc;")
        
        # Actualitza l'historial
        new_record = {
            "change": f"{current_text} → {new_text}",
            "date": new_date,
            "documents": record.get('documents', [])
        }
        self.history.setdefault(cell_id, []).append(new_record)
        
        # Actualitza els documents associats
        self.associatedDocuments[cell_id] = record.get('documents', []).copy()
        
        QMessageBox.information(self, "Restaurar canvi", "El canvi ha estat restaurat correctament.")

    def openAssociatedDocument(self, item):
        file_path = item.data(Qt.UserRole)
        if file_path:
            QDesktopServices.openUrl(QUrl.fromLocalFile(file_path))

    def showAssociatedDocumentsFromLineEdit(self, cell_id):
        dialog = QDialog(self)
        dialog.setWindowTitle(f"Documents associats per al camp {cell_id}")
        layout = QVBoxLayout(dialog)
        listWidget = QListWidget()
        associated = self.associatedDocuments.get(cell_id, [])
        if associated:
            for doc in associated:
                item = QListWidgetItem(f"{doc.get('name','No nom')} - {doc.get('timestamp','')}")
                item.setData(Qt.UserRole, doc.get('path'))
                listWidget.addItem(item)
        else:
            listWidget.addItem("No hi ha documents associats")
        layout.addWidget(listWidget)
        buttonBox = QDialogButtonBox(QDialogButtonBox.Ok)
        buttonBox.accepted.connect(dialog.accept)
        layout.addWidget(buttonBox)
        listWidget.itemClicked.connect(self.openAssociatedDocument)
        dialog.setLayout(layout)
        dialog.exec_()

    def refresh_table(self):
        for group in self.group_info:
            cell_id = group["widget"].cell_id
            if cell_id in self.associatedDocuments and self.associatedDocuments[cell_id]:
                group["widget"].label.setStyleSheet("background-color: #ccffcc;")
            else:
                group["widget"].label.setStyleSheet("")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

TypeError: fromLocalFile(localfile: Optional[str]): argument 1 has unexpected type 'dict'

SystemExit: 0

C:\Users\34761933D.GCB\AppData\Local\Programs\Python\Python313\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


import datetime
from PyQt5.QtWidgets import (
    QDialog, QVBoxLayout, QTableWidget, QTableWidgetItem, QLineEdit, QHBoxLayout,
    QPushButton, QComboBox, QDialogButtonBox, QScrollArea, QSizePolicy
)
from PyQt5.QtCore import Qt

class StatusOnDateDialog(QDialog):
    def __init__(self, parent, group_info, history, table):
        super().__init__(parent)
        self.setWindowTitle("Estat en una data")
        self.group_info = group_info
        self.history = history
        self.table = table

        layout = QVBoxLayout(self)
        
        # ComboBox per seleccionar la data
        date_combo = QComboBox()
        change_dates = sorted({record['date'] for history in self.history.values() for record in history})
        date_combo.addItems(change_dates)
        layout.addWidget(date_combo)
        
        buttonBox = QDialogButtonBox(QDialogButtonBox.Ok | QDialogButtonBox.Cancel)
        buttonBox.accepted.connect(self.accept)
        buttonBox.rejected.connect(self.reject)
        layout.addWidget(buttonBox)
        
        self.setLayout(layout)
        if self.exec_() == QDialog.Accepted:
            selected_date = date_combo.currentText()
            date_obj = datetime.datetime.strptime(selected_date, "%Y-%m-%d %H:%M:%S")
            self.show_status_on_date_table(date_obj)

    def show_status_on_date_table(self, date_obj):
        dialog = QDialog(self)
        dialog.setWindowTitle(f"Estat en la data {date_obj.strftime('%Y-%m-%d %H:%M:%S')}")
        dialog.setSizeGripEnabled(True)
        layout = QVBoxLayout(dialog)
        
        # Botó per maximitzar la finestra
        maximize_button = QPushButton("Maximitza")
        maximize_button.clicked.connect(dialog.showMaximized)
        layout.addWidget(maximize_button)
        
        # Afegir QScrollArea per a barres de desplaçament
        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        layout.addWidget(scroll_area)

        table = QTableWidget()
        table.setRowCount(self.table.rowCount())
        table.setColumnCount(self.table.columnCount())
        table.setHorizontalHeaderLabels([self.table.horizontalHeaderItem(i).text() for i in range(self.table.columnCount())])

        for group in self.group_info:
            cell_id = group["widget"].cell_id
            latest_validated_text = None
            earliest_change_text = None

            # Trobar l'últim text validat abans de la data
            for record in reversed(self.history.get(cell_id, [])):
                record_date = datetime.datetime.strptime(record['date'], "%Y-%m-%d %H:%M:%S")
                if record_date < date_obj:
                    latest_validated_text = record['change'].split("→")[-1].strip()
                    break

            # Trobar el primer canvi després de la data
            for record in self.history.get(cell_id, []):
                record_date = datetime.datetime.strptime(record['date'], "%Y-%m-%d %H:%M:%S")
                if record_date > date_obj:
                    earliest_change_text = record['change'].split("→")[-1].strip()
                    break

            row, col = map(int, cell_id.split('-'))
            if latest_validated_text:
                table.setItem(row, col * 2, QTableWidgetItem(latest_validated_text))
            if earliest_change_text:
                line_edit = QLineEdit()
                line_edit.setText(earliest_change_text)
                line_edit.setReadOnly(True)
                table.setCellWidget(row, col * 2 + 1, line_edit)

        # Configurar el menú contextual per a la taula de consulta
        table.setContextMenuPolicy(Qt.CustomContextMenu)
        table.customContextMenuRequested.connect(lambda pos: self.onContextMenuReadOnly(table, pos))

        # Afegir botons de selecció de columnes a la finestra de diàleg
        top_button_layout = QHBoxLayout()
        select_columns_button = QPushButton("Seleccionar columnes visibles")
        select_columns_button.clicked.connect(lambda: self.open_column_selection_dialog_for_table(table))
        top_button_layout.addWidget(select_columns_button)
        
        predefined_buttons = {}
        btn_inici = QPushButton("Inici")
        btn_inici.setCheckable(True)
        top_button_layout.addWidget(btn_inici)
        predefined_buttons[frozenset(COLUMNS_INICI)] = btn_inici
        
        btn_dades_entitat = QPushButton("Dades d'entitat")
        btn_dades_entitat.setCheckable(True)
        top_button_layout.addWidget(btn_dades_entitat)
        predefined_buttons[frozenset(COLUMNS_DADES_ENTITAT)] = btn_dades_entitat
        
        btn_particip = QPushButton("Partícips")
        btn_particip.setCheckable(True)
        top_button_layout.addWidget(btn_particip)
        predefined_buttons[frozenset(COLUMNS_PARTICIPS)] = btn_particip
        
        btn_membres = QPushButton("Membres")
        btn_membres.setCheckable(True)
        top_button_layout.addWidget(btn_membres)
        predefined_buttons[frozenset(COLUMNS_MEMBRES)] = btn_membres
        
        btn_persones = QPushButton("Persones")
        btn_persones.setCheckable(True)
        top_button_layout.addWidget(btn_persones)
        predefined_buttons[frozenset(COLUMNS_PERSONES)] = btn_persones
        
        for btn in predefined_buttons.values():
            btn.toggled.connect(lambda: self.update_predefined_columns_visibility_for_table(table, predefined_buttons))

        layout.addLayout(top_button_layout)
        scroll_area.setWidget(table)
        dialog.setLayout(layout)
        dialog.exec_()

    def open_column_selection_dialog_for_table(self, table):
        dialog = ColumnSelectionDialog(table, self)
        dialog.exec_()
        self.update_visible_rows_for_table(table)

    def update_predefined_columns_visibility_for_table(self, table, predefined_buttons):
        fixed_columns = ["0.Codi Catàleg", "0.Denominació"]
        active_predefined = set()
        for columns_set, btn in predefined_buttons.items():
            if btn.isChecked():
                active_predefined.update(columns_set)
        for col in range(table.columnCount()):
            header = table.horizontalHeaderItem(col).text().strip()
            if header in fixed_columns:
                table.setColumnHidden(col, False)
            else:
                visible = header in active_predefined if active_predefined else True
                table.setColumnHidden(col, not visible)
        self.update_visible_rows_for_table(table)

    def update_visible_rows_for_table(self, table):
        factor = 10
        num_columnes_visibles = sum(not table.isColumnHidden(col) for col in range(table.columnCount()))
        max_files = num_columnes_visibles * factor
        for row in range(1, table.rowCount()):  # Exclou la fila 0 (els filtres)
            table.setRowHidden(row, row >= max_files)
        self.apply_filters_for_table(table)

    def apply_filters_for_table(self, table):
        num_rows = table.rowCount()
        num_cols = table.columnCount()
        for row in range(1, num_rows):  # Comença per la fila 1 (les dades)
            show_row = True
            for col in range(num_cols):
                if not table.isColumnHidden(col):
                    filter_text = self.filter_edits[col].text().strip().lower()
                    if filter_text:
                        cell_text = ""
                        widget = table.cellWidget(row, col)
                        if widget is not None:
                            texts = []
                            if hasattr(widget, "label"):
                                texts.append(widget.label.text().strip().lower())
                            if hasattr(widget, "line_edit"):
                                texts.append(widget.line_edit.text().strip().lower())
                            cell_text = " ".join(texts)
                        else:
                            item = table.item(row, col)
                            if item is not None:
                                cell_text = item.text().strip().lower()
                        if filter_text not in cell_text:
                            show_row = False
                            break
            table.setRowHidden(row, not show_row)

    def onContextMenuReadOnly(self, table, pos):
        index = table.indexAt(pos)
        if not index.isValid():
            return
        row = index.row()
        col = index.column()
        if col % 2 == 0:
            cell_id = f"{row}-{col // 2}"
        else:
            cell_id = f"{row}-{(col-1) // 2}"
        from PyQt5.QtWidgets import QMenu
        menu = QMenu(self)
        actionPending = menu.addAction("Documents pendents")
        actionAssociated = menu.addAction("Documents associats")
        actionHistory = menu.addAction("Historial de canvis")
        action = menu.exec_(table.viewport().mapToGlobal(pos))
        if action == actionPending:
            show_pending_documents(self, cell_id)
        elif action == actionAssociated:
            self.showAssociatedDocumentsFromLineEdit(cell_id)
        elif action == actionHistory:
            self.showHistoryFromLabel(cell_id)

def show_status_on_date(parent, group_info, history, table):
    StatusOnDateDialog(parent, group_info, history, table)

import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QTableWidget, QPushButton,
    QVBoxLayout, QHBoxLayout, QWidget, QLineEdit, QDialog, QListWidget, QDialogButtonBox, QListWidgetItem, QMessageBox
)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QDesktopServices
import datetime

# Suposem que els següents mòduls estan disponibles:
# from custom_widgets import DoubleViewWidget
# from excel_handler import load_excel_and_populate_table
# from pending_documents_manager import show_pending_documents
# from history_manager import show_history
# from validation_manager import validate_modifications
# from session_manager import save_session, load_session
# from columns import COLUMNS_INICI, COLUMNS_DADES_ENTITAT, COLUMNS_PARTICIPS, COLUMNS_MEMBRES, COLUMNS_PERSONES
#from status_on_date import show_status_on_date  # Importar el mòdul creat

def toggle_changes_visibility(main_window, button):
    if not hasattr(main_window, "changes_visible"):
        main_window.changes_visible = False
    main_window.changes_visible = not main_window.changes_visible
    for group in main_window.group_info:
        group["widget"].line_edit.setVisible(main_window.changes_visible)
        if hasattr(group["widget"], "checkbox"):
            group["widget"].checkbox.setVisible(main_window.changes_visible)
    if main_window.changes_visible:
        button.setText("Amaga canvis")
    else:
        button.setText("Mostra canvis")

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Gestor de Documents Pendents")
        
        # Inicialitza els diccionaris d'estat
        self.pendingDocuments = {}
        self.associatedDocuments = {}
        self.history = {}
        self.excel_df = None  # Aquí es guardarà el contingut original de l'Excel
        
        central_widget = QWidget()
        main_layout = QVBoxLayout(central_widget)
        
        # Primera fila: botons de selecció de columnes (una sola fila)
        top_button_layout = QHBoxLayout()
        self.select_columns_button = QPushButton("Seleccionar columnes visibles")
        self.select_columns_button.clicked.connect(self.open_column_selection_dialog)
        top_button_layout.addWidget(self.select_columns_button)
        
        self.predefined_buttons = {}
        self.btn_inici = QPushButton("Inici")
        self.btn_inici.setCheckable(True)
        top_button_layout.addWidget(self.btn_inici)
        self.predefined_buttons[frozenset(COLUMNS_INICI)] = self.btn_inici
        
        self.btn_dades_entitat = QPushButton("Dades d'entitat")
        self.btn_dades_entitat.setCheckable(True)
        top_button_layout.addWidget(self.btn_dades_entitat)
        self.predefined_buttons[frozenset(COLUMNS_DADES_ENTITAT)] = self.btn_dades_entitat
        
        self.btn_particip = QPushButton("Partícips")
        self.btn_particip.setCheckable(True)
        top_button_layout.addWidget(self.btn_particip)
        self.predefined_buttons[frozenset(COLUMNS_PARTICIPS)] = self.btn_particip
        
        self.btn_membres = QPushButton("Membres")
        self.btn_membres.setCheckable(True)
        top_button_layout.addWidget(self.btn_membres)
        self.predefined_buttons[frozenset(COLUMNS_MEMBRES)] = self.btn_membres
        
        self.btn_persones = QPushButton("Persones")
        self.btn_persones.setCheckable(True)
        top_button_layout.addWidget(self.btn_persones)
        self.predefined_buttons[frozenset(COLUMNS_PERSONES)] = self.btn_persones
        
        for btn in self.predefined_buttons.values():
            btn.toggled.connect(self.update_predefined_columns_visibility)
        
        main_layout.addLayout(top_button_layout)
        
        # Fila de filtres: s'afegeix un QLineEdit per a cada columna a la primera fila de la taula
        self.filter_edits = []

        # Segona fila: botons de mostrar/amagar modificacions i validar, a l'esquerra
        bottom_button_layout = QHBoxLayout()
        bottom_button_layout.setAlignment(Qt.AlignLeft)
        self.validate_button = QPushButton("Validar modificacions")
        self.validate_button.clicked.connect(lambda: validate_modifications(self))
        bottom_button_layout.addWidget(self.validate_button)
        
        self.toggle_button = QPushButton("Mostrar / Ocultar canvis")
        self.toggle_button.clicked.connect(lambda: toggle_changes_visibility(self, self.toggle_button))
        bottom_button_layout.addWidget(self.toggle_button)
        
        self.date_status_button = QPushButton("Estat en una data")
        self.date_status_button.clicked.connect(lambda: show_status_on_date(self, self.group_info, self.history, self.table))
        bottom_button_layout.addWidget(self.date_status_button)
        
        main_layout.addLayout(bottom_button_layout)

        
        # Taula per mostrar les dades
        self.table = QTableWidget()
        main_layout.addWidget(self.table)
        
        self.setCentralWidget(central_widget)
        
        self.table.setContextMenuPolicy(Qt.CustomContextMenu)
        self.table.customContextMenuRequested.connect(self.onContextMenu)
        self.table.horizontalHeader().setStyleSheet("font-weight: bold; background-color: lightgrey;")
        
        self.group_info = []
        self.pendingDocuments = {}
        self.associatedDocuments = {}
        self.history = {}
        self.selectedChanges = set()
        self.changes_visible = False

        # Carrega l'Excel i omple la taula amb els widgets corresponents.
        load_excel_and_populate_table(self.table, self.group_info, self)
        # Afegeix la fila de filtres; aquesta fila s'insereix com la fila 0
        self.add_filter_row()
        
        # Amaga les caselles de modificació i també les de selecció (els widgets de les dades, no els filtres):
        for group in self.group_info:
            group["widget"].line_edit.hide()
            if hasattr(group["widget"], "checkbox"):
                group["widget"].checkbox.hide()

        self.update_visible_rows()
    
    def add_filter_row(self):
        """Insereix una fila de filtres a l'índex 0 de la taula, sota les capçaleres."""
        self.table.insertRow(0)
        self.filter_edits = []
        num_cols = self.table.columnCount()
        for col in range(num_cols):
            header_item = self.table.horizontalHeaderItem(col)
            header_text = header_item.text().strip() if header_item else f"Col {col+1}"
            edit = QLineEdit()
            edit.setPlaceholderText(f"Filtrar {header_text}")
            edit.textChanged.connect(self.apply_filters)
            self.table.setCellWidget(0, col, edit)
            self.filter_edits.append(edit)
    
    def apply_filters(self):
        """
        Aplica els filtres de la fila 0 a les files de dades (a partir de la fila 1).
        Per cada cel·la de cada fila visible, es combina el text del widget (si n'hi ha) 
        (tant el de l'etiqueta com el de la QLineEdit) o el text de l'item, i si el filtre no
        es troba en aquest text, la fila es fa invisible.
        """
        num_rows = self.table.rowCount()
        num_cols = self.table.columnCount()
        for row in range(1, num_rows):  # Comença per la fila 1 (les dades)
            show_row = True
            for col in range(num_cols):
                if not self.table.isColumnHidden(col):
                    filter_text = self.filter_edits[col].text().strip().lower()
                    if filter_text:
                        cell_text = ""
                        widget = self.table.cellWidget(row, col)
                        if widget is not None:
                            texts = []
                            if hasattr(widget, "label"):
                                texts.append(widget.label.text().strip().lower())
                            if hasattr(widget, "line_edit"):
                                texts.append(widget.line_edit.text().strip().lower())
                            cell_text = " ".join(texts)
                        else:
                            item = self.table.item(row, col)
                            if item is not None:
                                cell_text = item.text().strip().lower()
                        if filter_text not in cell_text:
                            show_row = False
                            break
            self.table.setRowHidden(row, not show_row)
    
    def open_column_selection_dialog(self):
        dialog = ColumnSelectionDialog(self.table, self)
        dialog.exec_()
        self.update_visible_rows()
        
    def update_predefined_columns_visibility(self):
        fixed_columns = ["0.Codi Catàleg", "0.Denominació"]
        active_predefined = set()
        for columns_set, btn in self.predefined_buttons.items():
            if btn.isChecked():
                active_predefined.update(columns_set)
        for col in range(self.table.columnCount()):
            header = self.table.horizontalHeaderItem(col).text().strip()
            if header in fixed_columns:
                self.table.setColumnHidden(col, False)
            else:
                visible = header in active_predefined if active_predefined else True
                self.table.setColumnHidden(col, not visible)
        self.update_visible_rows()
    
    def update_visible_rows(self):
        factor = 10
        num_columnes_visibles = sum(not self.table.isColumnHidden(col) for col in range(self.table.columnCount()))
        max_files = num_columnes_visibles * factor
        for row in range(1, self.table.rowCount()):  # Exclou la fila 0 (els filtres)
            self.table.setRowHidden(row, row >= max_files)
        self.apply_filters()

    def onSelectionChanged(self, cell_id, selected):
        if selected:
            self.selectedChanges.add(cell_id)
        else:
            self.selectedChanges.discard(cell_id)
    
    def onContextMenu(self, pos):
        index = self.table.indexAt(pos)
        # Si es fa clic a la fila de filtres, no es mostra cap menú
        if index.row() == 0:
            return
        if not index.isValid():
            return
        row = index.row()
        if index.column() % 2 == 0:
            cell_id = f"{row}-{index.column() // 2}"
        else:
            cell_id = f"{row}-{(index.column()-1) // 2}"
        from PyQt5.QtWidgets import QMenu
        menu = QMenu(self)
        actionPending = menu.addAction("Documents pendents")
        actionAssociated = menu.addAction("Documents associats")
        actionHistory = menu.addAction("Historial de canvis")
        action = menu.exec_(self.table.viewport().mapToGlobal(pos))
        if action == actionPending:
            show_pending_documents(self, cell_id)
        elif action == actionAssociated:
            self.showAssociatedDocumentsFromLineEdit(cell_id)
        elif action == actionHistory:
            self.showHistoryFromLabel(cell_id)
    
    def setCellBackground(self, cell_id, color):
        try:
            row, col = map(int, cell_id.split('-'))
        except Exception:
            return
        from PyQt5.QtGui import QBrush
        item = self.table.item(row, col * 2)
        if item:
            item.setBackground(QBrush(color))
    
    def associateDocumentFromLineEdit(self, cell_id):
        from PyQt5.QtWidgets import QFileDialog, QMessageBox
        import os, datetime
        filePath, _ = QFileDialog.getOpenFileName(self, "Selecciona Document", "")
        if filePath:
            fileName = os.path.basename(filePath)
            doc = {
                "path": filePath,
                "name": fileName,
                "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            self.associatedDocuments.setdefault(cell_id, []).append(doc)
            QMessageBox.information(self, "Document Associat", f"Document '{fileName}' associat correctament al camp {cell_id}.")
    
    def showPendingDocumentsFromLineEdit(self, cell_id):
        show_pending_documents(self, cell_id)
    
    def showHistoryFromLabel(self, cell_id):
        dialog = QDialog(self)
        dialog.setWindowTitle(f"Historial de canvis per al camp {cell_id}")
        layout = QVBoxLayout(dialog)
        listWidget = QListWidget()
        records = self.history.get(cell_id, [])
        if records:
            for record in records:
                item = QListWidgetItem(f"{record['date']}: {record['change']}")
                item.setData(Qt.UserRole, record)
                listWidget.addItem(item)
                if 'documents' in record:
                    for doc in record['documents']:
                        doc_item = QListWidgetItem(f"  Document: {doc['name']}")
                        doc_item.setData(Qt.UserRole, doc['path'])
                        listWidget.addItem(doc_item)
        else:
            listWidget.addItem("No hi ha canvis per aquest camp")
        layout.addWidget(listWidget)
        
        restore_button = QPushButton("Restaurar canvi")
        restore_button.clicked.connect(lambda: self.restore_change(cell_id, listWidget))
        layout.addWidget(restore_button)
        
        buttonBox = QDialogButtonBox(QDialogButtonBox.Ok)
        buttonBox.accepted.connect(dialog.accept)
        layout.addWidget(buttonBox)
        
        listWidget.itemClicked.connect(self.openAssociatedDocument)
        dialog.setLayout(layout)
        dialog.exec_()

    def restore_change(self, cell_id, listWidget):
        selected_items = listWidget.selectedItems()
        if not selected_items:
            QMessageBox.warning(self, "Selecció", "Selecciona un canvi per restaurar.")
            return
        selected_item = selected_items[0]
        record = selected_item.data(Qt.UserRole)
        if not record:
            QMessageBox.warning(self, "Selecció", "Selecciona un canvi vàlid per restaurar.")
            return

        # Obtenir l'últim text abans de la restauració
        current_text = None
        for group in self.group_info:
            if group["widget"].cell_id == cell_id:
                current_text = group["widget"].label.text()
                break
        if current_text is None:
            QMessageBox.warning(self, "Error", "No s'ha pogut trobar el text actual.")
            return
        
        new_text = record['change'].split("→")[-1].strip()
        new_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # Actualitza el text i els documents associats
        for group in self.group_info:
            if group["widget"].cell_id == cell_id:
                group["widget"].label.setText(new_text)
                group["widget"].line_edit.setText(new_text)
                group["widget"].label.setStyleSheet("background-color: #ccffcc;")
        
        # Actualitza l'historial
        new_record = {
            "change": f"{current_text} → {new_text}",
            "date": new_date,
            "documents": record.get('documents', [])
        }
        self.history.setdefault(cell_id, []).append(new_record)
        
        # Actualitza els documents associats
        self.associatedDocuments[cell_id] = record.get('documents', []).copy()
        
        QMessageBox.information(self, "Restaurar canvi", "El canvi ha estat restaurat correctament.")

    def openAssociatedDocument(self, item):
        file_path = item.data(Qt.UserRole)
        if file_path:
            QDesktopServices.openUrl(QUrl.fromLocalFile(file_path))

    def showAssociatedDocumentsFromLineEdit(self, cell_id):
        dialog = QDialog(self)
        dialog.setWindowTitle(f"Documents associats per al camp {cell_id}")
        layout = QVBoxLayout(dialog)
        listWidget = QListWidget()
        associated = self.associatedDocuments.get(cell_id, [])
        if associated:
            for doc in associated:
                item = QListWidgetItem(f"{doc.get('name','No nom')} - {doc.get('timestamp','')}")
                item.setData(Qt.UserRole, doc.get('path'))
                listWidget.addItem(item)
        else:
            listWidget.addItem("No hi ha documents associats")
        layout.addWidget(listWidget)
        buttonBox = QDialogButtonBox(QDialogButtonBox.Ok)
        buttonBox.accepted.connect(dialog.accept)
        layout.addWidget(buttonBox)
        listWidget.itemClicked.connect(self.openAssociatedDocument)
        dialog.setLayout(layout)
        dialog.exec_()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())